# 0.Building a VAR model in Python

The procedure to build a VAR model involves the following steps:

1. Analyze the time series characteristics
2. Test for causation amongst the time series
3. Test for stationarity
4. Transform the series to make it stationary, if needed
5. Find optimal order (p)
6. Prepare training and test datasets
7. Train the model
8. Roll back the transformations, if any.
9. Evaluate the model using test set
10. Forecast to future


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install statsmodels

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic


import math
import os
import time
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from copy import deepcopy
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.stattools import acf

from fbprophet import Prophet
import warnings
import datetime
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings(action='ignore')
from collections import defaultdict
import gc

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
cd drive/MyDrive/Dacon/BitTrader

/content/drive/MyDrive/Dacon/BitTrader


In [ ]:
train_x_df = pd.read_csv( "./data/train_x_df.csv")
train_y_df = pd.read_csv( "./data/train_y_df.csv")
test_x_df = pd.read_csv( "./data/test_x_df.csv")
submission = pd.read_csv("./data/sample_submission.csv")

In [ ]:
set(train_x_df.coin_index)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

# 4.Data Preprocessing

In [ ]:
def adjust(val, length= 6): return str(val).ljust(length)

In [ ]:
def df2d_to_array3d(df_2d):
    # 안쓰고 싶은 feature가 있다면 다음 목록에서 빼기
    features = ['coin_index', 'open', 'high', 'low', 'close','volume', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av']
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    nfeatures = len(features)
    time_size = len(set(df_2d.time))
    sample_size = len(set(df_2d.sample_id))
    sample_index = set(train_x_df.sample_id)
    array_3d = df_2d.loc[:,features].values.reshape([sample_size, time_size, nfeatures])
    return array_3d

In [ ]:

def df_var(arr,idx):
    start_time = '2021-01-31 00:00:00'
    start_dt = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

    features = ['coin_index', 'open', 'high', 'low', 'close','volume', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av']
    # 1 : open, 
    # 2 : high, 
    # 3 : low, ...
    feature_dict = {i:feature for i,feature in enumerate(features)}

    df = pd.DataFrame(index=[start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, arr.shape[1]+1).tolist()])
    df.index.name = 'date'

    for feature_idx in range(1,arr.shape[-1]):
        x_series = arr[idx,:,feature_idx]
        df[feature_dict[feature_idx]] = x_series.tolist()
    return df

In [ ]:
## 정상성을 판단하여 차분해야하는지 검정해주는 함수
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # # Print Summary
    # print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    # print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    # print(f' Significance Level    = {signif}')
    # print(f' Test Statistic        = {output["test_statistic"]}')
    # print(f' No. Lags Chosen       = {output["n_lags"]}')

    # for key,val in r[4].items():
    #     print(f' Critical value {adjust(key)} = {round(val, 3)}')

    # if p_value <= signif:
    #     print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
    #     print(f" => Series is Stationary.")
    # else:
    #     print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
    #     print(f" => Series is Non-Stationary.")    
    return p_value

In [ ]:
def invert_transformation(df_train, df_forecast):
    df_fc = pd.DataFrame()
    for col in df_forecast.columns:
        if col.endswith('_1d'):
            df_fc[col.replace('_1d','_forecast')] = df_train[col.replace('_1d','')].iloc[-1] + df_forecast[col].cumsum()
        else:
            df_fc[f'{col}_forecast'] = df_forecast[col]
    return df_fc

In [ ]:
def forecast_accuracy(forecast, actual):
    mae = np.mean(np.abs(forecast - actual))    # MAE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    return {
            'mae': mae, 
            'rmse':rmse,
            }

In [ ]:
gc.collect()

500

In [ ]:
# coin index별 샘플 갯수
train_x_df.groupby('coin_index').sample_id.apply(lambda x: len(set(x)))

coin_index
0     907
1     382
2     101
3     129
4     930
5     545
6    1065
7     947
8    1178
9    1178
Name: sample_id, dtype: int64

In [ ]:
final_predict_collections_per_coin = defaultdict(list)  # 샘플아이디 각각을 key로 가짐, value : 각 모델들의 해당 샘플아이디에 대한 예측결과 array들을 모아놓음
# 얘는 각 샘플별로 수많은 prediction 결과들을 가지고 있게 되는 최종 결과가 될 것임.
# 각 샘플 아이디 내부의 여러 모델들로부터 나온 여러 prediction들을 평균냄으로써 앙상블 결과를 각 샘플아이디 별로 가지게 됨
# final_predict_collections_per_coin 이 dictionary의 각각 샘플 아이디 별로 np.mean(axis=0) 해주면 됨


for coin_index in tqdm(range(0,10)):
    ## coin_index별로 train 뽑기
    # coin_index 는 0~9
    # coin_index = 0

    train_x_array = df2d_to_array3d(train_x_df)
    train_y_array = df2d_to_array3d(train_y_df)
    train_array = np.concatenate([train_x_array,train_y_array],axis=1)

    # 테스트셋
    test_x_array = df2d_to_array3d(test_x_df)
    test_sample_idx_current_coin = list(set(test_x_df.loc[test_x_df.coin_index==coin_index,:].sample_id))    # 테스트셋 중 현재 코인인덱스에 해당하는 샘플아이디들을 모아놓은 리스트


    print('='*100)
    print(f'coin_index : {coin_index}')

    # groupby_coin_last_open_value = train_x_df.loc[train_x_df.coin_index==coin_index,:].groupby(by='sample_id').open.apply(lambda x: list(x)[-2])
    # last_open_max_index = groupby_coin_last_open_value.index[groupby_coin_last_open_value.argmax()]
    idxes = train_x_df.loc[train_x_df.coin_index==coin_index,:].groupby(by='sample_id').open.max().index.tolist()
    
    # idxes 모두에 대해서 모델을 적합하기에는 시간이 너무 오래걸려서 일부만 (5%정도)만 모델을 만들었습니다.
    # import random
    # random.seed(123)
    # idxes = random.sample(idxes,len(idxes)//20)
    # nsamples = 10
    # idxes = random.sample(idxes,nsamples)


    # print(f"coin_index=={coin_index}의 샘플들 중 모델링할 샘플은 {len(idxes)}개입니다")
    # idxes = range(train_array.shape[0])
    for sample_id in tqdm(idxes):
        # print(f'sample_id : {sample_id}')

        ## 원하는 sample_id로만 train_x & y 를 구축
        df = df_var(train_array,sample_id)

        # train의 y 120개를 Validation set으로 두고 모델링을 해보자.
        nobs = 120  
        df_train, df_test = df[:-nobs], df[-nobs:]

        # ADF Test on each column
        adf_pvalue_dict={}

        for name, column in df_train.iteritems():
            p_value = adfuller_test(series=column, name=column.name)
            adf_pvalue_dict[name]=p_value

        # 귀무가설 기각 못한 : Non-stationary features
        need_diff_features_1d = [name for name,pvalue in adf_pvalue_dict.items() if pvalue>0.05]
        # print(f'need_diff_features_1d : {need_diff_features_1d}')

        ## 1차 차분
        # 1st difference
        # non stationary feature들에 대해서만 차분을 진행해보자.
        df_differenced = deepcopy(df_train)
        df_differenced.loc[:,need_diff_features_1d] = df_differenced.loc[:,need_diff_features_1d].diff()
        df_differenced = df_differenced.dropna()
        # After 1st Differences, One More ADF Test on each column
        adf_pvalue_dict={}
        # ADF Test on each column of 1st Differences Dataframe
        for name, column in df_differenced.iteritems():
            p_value = adfuller_test(column, name=column.name)
            adf_pvalue_dict[name]=p_value
            # print('\n')

        ## 2차 차분
        # 1차 차분 뒤에도 귀무가설 기각 못한 : Non-stationary features
        # need_diff_features_2d = [name for name,pvalue in adf_pvalue_dict.items() if pvalue>0.05]
        # print(need_diff_features_2d)   # 이제 없음 # 차분 1번만에 모두 stationary
        # df_differenced.loc[:,need_diff_features_2d] = df_differenced.loc[:,need_diff_features_2d].diff()
        # df_differenced = df_differenced.dropna()

        ## How to Select the Order (P) of VAR model
        # 최대 p의 범위는 120으로 지정해놓고 Least AIC
        model = VAR(df_differenced)
        # print('='*20,'Choose P','='*20)
        # AIC_dict = {}
        maxlags = 60*2

        #################################################
        ########## 120으로 그냥 통일해서 해봤음 ############
        ### 가장 적절한 P 값을 찾는 부분

        # for i in range(1,maxlags+1):
        #     result = model.fit(i)
        #     AIC_dict[i] = result.aic
        ## 이부분의 시간 소요가 너무 많다..

        top_idx = 0 # top0 : AIC 제일 좋을 때의 p값 (AIC가 제일 작을 때)

        # Train the VAR Model of Selected Order(p)
        # best_AIC_p = sorted(AIC_dict.items(),key=lambda x: x[-1])[top_idx][0]
        # best_p = best_AIC_p
        best_p = 120
        model_fitted = model.fit(best_p)

        ## Check for Serial Correlation of Residuals (Errors) using Durbin Watson Statistic
        ## 더빈 왓슨 통계량은 0 ~ 4사이의 값을 갖을 수 있음
        #0에 가까울수록 → 양의 상관관계
        #4에 가까울수록 → 음의 상관관계
        #2에 가까울수록 → 오차항의 자기상관이 없음
        # lags = 120
        # from statsmodels.stats.stattools import durbin_watson
        # out = durbin_watson(model_fitted.resid)

        # for col, val in zip(df.columns, out):
        #     print(adjust(col), ':', round(val, 2))

        ## How to Forecast VAR model using statsmodels
        # Get the lag order
        lag_order = model_fitted.k_ar

        ## Input data for forecasting
        # forecast_input = df_differenced.values[-lag_order:]
        # print(forecast_input.shape)


        #### Valid Forecast ####
        diff_column_names = [f'{c}_1d' if c in need_diff_features_1d else c for c in df.columns]
        # fc = model_fitted.forecast(y=forecast_input, steps=nobs)
        # df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=diff_column_names)

        ### 역변환 (차분을 했다면)
        # df_results = invert_transformation(df_train, df_forecast)

        # validation 정확도 보기
        # val_accuracy_prod = forecast_accuracy(df_results[f'open_forecast'].values, df_test['open'])
        # val_mae = val_accuracy_prod['mae']
        # val_rmse = val_accuracy_prod['rmse']
        # if val_rmse > 0.5: break

        # model들을 해당 coin_index를 key로 가지는 dict에 append로 저장


        # print(f"sample_id=={sample_id}로 학습시킨 모델로 coin_index=={coin_index}인 test set의 샘플들에 각각 prediction을 실시합니다")
        # print(f"best_p : {best_p} Model입니다.")
        # coin_index == coin_index에 대한 모델이므로 해당 coin index의 test x들만 뽑아서 각각 prediction 결과를 만든다.
        for test_sample_idx in test_sample_idx_current_coin:
        # for test_sample_idx in tqdm(test_sample_idx_current_coin):
            test_x_idx_var = df_var(test_x_array,test_sample_idx)
            test_x_differenced = deepcopy(test_x_idx_var)
            test_x_differenced.loc[:,need_diff_features_1d] = test_x_differenced.loc[:,need_diff_features_1d].diff()
            test_x_differenced.dropna(inplace=True)
            test_forecast_input = test_x_differenced.values[-lag_order:]
            test_fc = model_fitted.forecast(y=test_forecast_input, steps=nobs)
            test_forecast = pd.DataFrame(test_fc, index=range(nobs), columns=diff_column_names) # 예측결과 test_fc를 test_forecast라는 데이터프레임화 (역변환 인풋을 위해)
            test_results = invert_transformation(test_x_idx_var, test_forecast)['open_forecast'] # 예측결과 역변환, 우리의 타겟인 open_forecast만 뽑습니다.
            test_results = np.array(test_results) # coin_index 내의 어떤 한 샘플 아이디로 모델적합을 했고,  test predict를 한 결과이다.

            final_predict_collections_per_coin[test_sample_idx].append(test_results) # 어떤 한 샘플 아이디의 prediction 결과를 해당 샘플 아이디에 append 
                                                                                    # (여러 모델들의 예측결과가 차곡차곡 모일 것이다.)

coin_index : 0



coin_index : 1



coin_index : 2



coin_index : 3



coin_index : 4



coin_index : 5



coin_index : 6



coin_index : 7



coin_index : 8



coin_index : 9


In [ ]:
print(max(final_predict_collections_per_coin.keys()))
print(min(final_predict_collections_per_coin.keys()))
print(len(final_predict_collections_per_coin.keys()))

528
0
529


# 17.Submission

In [ ]:
# 여러개의 모델 output들을 지니고 있는 샘플 아이디 각각에 대해서 np.mean을 적용함으로써 앙상블결과 도출
prediction_after_ensemble_per_test_id = {k:np.mean(np.array(v),axis=0) for k,v in final_predict_collections_per_coin.items()}
# len(prediction_after_ensemble_per_test_id.keys())

# submission 파일 만들기
final_output = pd.DataFrame( 
                            # 샘플아이디 기준으로 정렬
                            sorted(
                                [
                                [k, v.max(), v.argmax()] 
                                for k,v in prediction_after_ensemble_per_test_id.items()
                                ], 
                                key=lambda x: x[0]),
                            columns=['sample_id','buy_quantity','sell_time'] 
                            ) 
final_output

,sample_id,buy_quantity,sell_time
0,0,1.860042e+00,119
1,1,1.224311e+00,105
2,2,1.531320e+07,119
3,3,2.716271e+00,118
4,4,1.277883e+00,59
...,...,...,...
524,524,6.369490e+03,119
525,525,6.464189e-01,0
526,526,1.057587e+00,13
527,527,3.890598e+00,119


In [ ]:
threshold = 60e+04
# threshold = 4 # 4로 해봤는데 안좋음.. 5로 한번 해볼까?

submission = final_output.copy()
submission['buy_quantity'] = submission['buy_quantity'].apply(lambda x: 1 if x>threshold else 0)
submission['buy_quantity'].value_counts()

0    507
1     22
Name: buy_quantity, dtype: int64

In [ ]:
submission.loc[submission.buy_quantity==1,:].sell_time.value_counts()

119    22
Name: sell_time, dtype: int64

In [ ]:
threshold = 6
# threshold = 4 # 4로 해봤는데 안좋음.. 5로 한번 해볼까?

submission = final_output.copy()
submission['buy_quantity'] = submission['buy_quantity'].apply(lambda x: 1 if x>threshold else 0)
submission['buy_quantity'].value_counts()

0    507
1     22
Name: buy_quantity, dtype: int64

In [ ]:
submission

,sample_id,buy_quantity,sell_time
0,0,0,119
1,1,0,105
2,2,1,119
3,3,0,118
4,4,0,59
...,...,...,...
524,524,0,119
525,525,0,0
526,526,0,13
527,527,0,119


In [ ]:
# submission 파일 생성
from datetime import datetime
today_date = datetime.today().strftime("%m-%d")

import os 
lag_order = model_fitted.k_ar


folder_path = f"{today_date}_coinindex별_각각_모든샘플_앙상블_threshold_{threshold}_1380개train_x만_사용"
if not os.path.isdir(f"./submission_VAR/{folder_path}"):
    os.makedirs(f"./submission_VAR/{folder_path}")

submission_path = f"./submission_VAR/{folder_path}/{folder_path}_submission"

submission.to_csv(f'{submission_path}.csv',index=False)

# .Conclusion
In this article we covered VAR from scratch beginning from the intuition behind it, interpreting the formula, causality tests, finding the optimal order of the VAR model, preparing the data for forecasting, build the model, checking for serial autocorrelation, inverting the transform to get the actual forecasts, plotting the results and computing the accuracy metrics.

Hope you enjoyed reading this as much as I did writing it. I will see you in the next one.



In [ ]:
best_submission = pd.read_csv(".//submission_VAR/03-27_coinindex별_각각10앙상블_threshold_6_1380개train_x만_사용/03-27_coinindex별_각각10앙상블_threshold_6_1380개train_x만_사용_submission.csv")

In [ ]:
best_submission.loc[best_submission.buy_quantity==1,:].sell_time.value_counts()

119    3
117    3
116    3
115    2
114    2
108    2
78     1
118    1
112    1
110    1
104    1
103    1
96     1
Name: sell_time, dtype: int64

In [ ]:
best_submission_time = 96
a = set(best_submission.loc[best_submission.buy_quantity==1,:].loc[best_submission.sell_time==best_submission_time,:].sample_id.value_counts().index)
b = set(submission.loc[submission.buy_quantity==1,:].loc[submission.sell_time==119,:].sample_id.value_counts().index)
a&b

set()